In [16]:
require(data.table)
library(Hmisc)

Warning message:
"package 'Hmisc' was built under R version 3.6.3"Loading required package: lattice
Loading required package: survival
Warning message:
"package 'survival' was built under R version 3.6.3"Loading required package: Formula
Warning message:
"package 'Formula' was built under R version 3.6.3"Loading required package: ggplot2
Warning message:
"package 'ggplot2' was built under R version 3.6.3"
Attaching package: 'Hmisc'

The following objects are masked from 'package:base':

    format.pval, units



In [5]:
train = fread('train_raw.csv', encoding = "UTF-8")

In [6]:
head(train)

time_stamp,contentid,user_action,sellingprice,product_name,brand_id,brand_name,businessunit,product_gender,category_id,Level1_Category_Id,Level1_Category_Name,Level2_Category_Id,Level2_Category_Name,Level3_Category_Id,Level3_Category_Name,gender,unique_id,type
2020-12-02T22:26:14.023Z,39918893,favorite,3099.00,PerfectCare 600 EW6F449ST A+++ 9 KG 1400 Devir Çamasir Makinesi,8511,Electrolux,Beyaz Esya,Unisex,1272,1071,Elektronik,1212,Beyaz Esya,1272,Çamasir Makinesi,F,425,train
2020-12-08T23:15:04.603Z,3558544,favorite,3079.00,WW90J5475FW A+++ 1400 Devir 9 kg Çamasir Makinesi,3228,Samsung,Beyaz Esya,,1272,1071,Elektronik,1212,Beyaz Esya,1272,Çamasir Makinesi,F,425,train
2020-12-05T16:19:01.157Z,31292729,favorite,3999.00,KM 9711 A++ 9 kg Çamasir Kurutma Makinesi,10989,Vestel,Beyaz Esya,Unisex,1276,1071,Elektronik,1212,Beyaz Esya,1276,Kurutma Makinesi,F,425,train
2020-12-05T16:28:00Z,6363103,visit,2544.00,CMI 9710 A+++ 1000 Devir 9 kg Çamasir Makinesi,10989,Vestel,Beyaz Esya,,1272,1071,Elektronik,1212,Beyaz Esya,1272,Çamasir Makinesi,F,425,train
2020-12-02T22:26:59Z,39918893,visit,3099.00,PerfectCare 600 EW6F449ST A+++ 9 KG 1400 Devir Çamasir Makinesi,8511,Electrolux,Beyaz Esya,Unisex,1272,1071,Elektronik,1212,Beyaz Esya,1272,Çamasir Makinesi,F,425,train
2020-11-03T21:04:11Z,32593071,visit,266.65,Siyah Kadin Abiye Ayakkabi 01AYH158420A100,59,Hotiç,Branded Shoes A,Kadin,431,403,Ayakkabi,430,Topuklu Ayakkabi,431,Abiye Ayakkabi,F,425,train


In [7]:
nrow(train)

[1] 5493268

In [12]:
str(train)

Classes 'data.table' and 'data.frame':	5493268 obs. of  19 variables:
 $ time_stamp          : chr  "2020-12-02T22:26:14.023Z" "2020-12-08T23:15:04.603Z" "2020-12-05T16:19:01.157Z" "2020-12-05T16:28:00Z" ...
 $ contentid           : int  39918893 3558544 31292729 6363103 39918893 32593071 4946868 6363103 6363100 6363100 ...
 $ user_action         : chr  "favorite" "favorite" "favorite" "visit" ...
 $ sellingprice        : num  3099 3079 3999 2544 3099 ...
 $ product_name        : chr  "PerfectCare 600 EW6F449ST A+++ 9 KG 1400 Devir Çamasir Makinesi" "WW90J5475FW A+++ 1400 Devir 9 kg Çamasir Makinesi" "KM 9711 A++ 9 kg Çamasir Kurutma Makinesi" "CMI 9710 A+++ 1000 Devir 9 kg Çamasir Makinesi" ...
 $ brand_id            : int  8511 3228 10989 10989 8511 59 1802 10989 10989 10989 ...
 $ brand_name          : chr  "Electrolux" "Samsung" "Vestel" "Vestel" ...
 $ businessunit        : chr  "Beyaz Esya" "Beyaz Esya" "Beyaz Esya" "Beyaz Esya" ...
 $ product_gender      : chr  "Unisex" "" "Unis

In [13]:
u_train = unique(train)

In [10]:
nrow(u_train)

[1] 2077356

In [15]:
length(unique(u_train$unique_id))

[1] 5618

In [17]:
describe(u_train)

u_train 

 19  Variables      2077356  Observations
--------------------------------------------------------------------------------
time_stamp 
       n  missing distinct 
 2077356        0  1583129 

lowest : 2020-10-14T10:42:23Z 2020-10-14T10:42:27Z 2020-10-14T10:42:31Z 2020-10-14T10:42:35Z 2020-10-14T10:42:38Z
highest: 2020-12-09T13:29:44Z 2020-12-09T13:29:47Z 2020-12-09T13:29:50Z 2020-12-09T13:29:52Z 2020-12-09T13:29:55Z
--------------------------------------------------------------------------------
contentid 
       n  missing distinct     Info     Mean      Gmd      .05      .10 
 2077354        2   482796        1 37987215 17813687  3034276  4732656 
     .25      .50      .75      .90      .95 
32589936 42259581 49701645 55084743 57547598 

lowest :    48852    51969    51973    51995    52007
highest: 67562911 67672020 67672059 67717726 67806253
--------------------------------------------------------------------------------
user_action 
       n  missing distinct 
 2077356 

In [18]:
customers = unique(u_train[, c("unique_id", "gender")])

In [20]:
table(customers$gender)


   F    M 
3679 1939 

* 5918 users, 3679 of which are female and 1939 of which are male (Class imbalance)
* Time of the actions starts from 2020-10-14 end at 2020-12-09, which is approximately 2 months
* 5 distinct actions: Adding the product to basket, adding the product to favorites, ordering the product, searching the product and visiting the product. Most records are from searching and visiting.
* Selling price of the product, ranging from 0 to 226,320 with 32,013 missing values.
* 3 level category information with id and character values, 2184 missing values.
* Gender of the product, Male/Female/Unisex, 257,333 missing values
* Business Unit, high level category information, 2184 missing values.
* Brand id and name, 2184 missing values.
* Product name, cannot be used in predictive models, only for descriptive analysis and observation, 2184 missing values.
* Content id, 

In [21]:
3679 / 5618

[1] 0.6548594